In [54]:
from IPython.core.debugger import set_trace

# %load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import csv
plt.style.use(style="seaborn")
%matplotlib inline

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train.csv', escapechar = "\\",quoting=csv.QUOTE_NONE)

In [ ]:
train['DESCRIPTION'] = train['DESCRIPTION'].fillna('a')
train['TITLE'] = train['TITLE'].fillna('a')
train['BULLET_POINTS'] = train['BULLET_POINTS'].fillna('a')
train['BRAND'] = train['BRAND'].fillna('a')
train.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,a,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),a,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,a,a,a,4


In [ ]:
train["FEATURE"] = train["TITLE"] + train["DESCRIPTION"] + train['BULLET_POINTS'] + train['BRAND']

In [ ]:
train.drop(["TITLE","DESCRIPTION",'BULLET_POINTS','BRAND'], axis = 1, inplace= True)
train.head()

,BROWSE_NODE_ID,FEATURE
0,0,"Pete The Cat Bedtime Blues Doll, 14.5 InchPete..."
1,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ..."
2,2,The Ultimate Self-Sufficiency Handbook: A Comp...
3,3,Amway Nutrilite Kids Chewable Iron Tablets (10...
4,4,Teacher Planner Company A4 6 Lesson Academic T...


In [ ]:
import string


def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
train["FEATURE"] = train.FEATURE.map(lambda x: remove_punct(x))

In [ ]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

train["FEATURE"] = train["FEATURE"].map(remove_stopwords)

In [ ]:
train.FEATURE

0          pete cat bedtime blues doll 145 inchpete cat c...
1          new yorker nyhm014 refrigerator magnet 2 x 35t...
2          ultimate selfsufficiency handbook complete gui...
3          amway nutrilite kids chewable iron tablets 100...
4          teacher planner company a4 6 lesson academic t...
                                 ...                        
2903019    premium aviator sunglasses hd polarized bright...
2903020    social distance stickers set 5 sticker slip re...
2903021    torrto face shield pack 5 adjustable elastic s...
2903022    typec 35 mm oppo r17 pro typec 35 mm audio jac...
2903023    kng orange polo tshirt 38performance inspired ...
Name: FEATURE, Length: 2903024, dtype: object

In [ ]:
groups = train.groupby('BROWSE_NODE_ID').count()

In [ ]:
groups[groups['FEATURE']>5000]

,FEATURE
BROWSE_NODE_ID,
4,34177
5,70318
11,21660
22,9020
40,14784
...,...
1922,7753
1971,5919
1988,5028


In [ ]:
trainA = train[train['BROWSE_NODE_ID']>50000]
print(len(train))
print(len(trainA))

2903024
90182


In [ ]:
def create_corpus_tk(df):
    corpus = []
    for text in df["FEATURE"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [ ]:
corpus = create_corpus_tk(trainA)

In [ ]:
train.to_csv('/content/drive/MyDrive/pre_processed_train.csv')

In [ ]:
from google.colab import files
files.download('pre_processed_train.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
corpus[0][:10]

['anbau',
 '4',
 'digit',
 'hand',
 'tally',
 'counter',
 'digital',
 'pitch',
 'counters',
 'clicker']

In [ ]:
train_size = int(trainA.shape[0] * 0.8)

train_sentences = trainA.FEATURE[:train_size]
train_labels = trainA.BROWSE_NODE_ID[:train_size]

test_sentences = trainA.FEATURE[train_size:]
test_labels = trainA.BROWSE_NODE_ID[train_size:]

In [ ]:
num_words = len(corpus)
print(num_words)

90182


In [56]:
# tokenizer = Tokenizer(num_words=num_words)
# tokenizer.fit_on_texts(train_sentences)

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(list(train_sentences)+list(test_sentences))
train_X = tokenizer.texts_to_sequences(train_sentences)
test_X = tokenizer.texts_to_sequences(train_sentences)

In [57]:
# train_sequences = tokenizer.texts_to_sequences(train_sentences)
# train_sentences

In [ ]:
# train_sequences[0]

In [66]:
max_len = 250
train_padded = pad_sequences(train_X, maxlen=max_len)
test_padded = pad_sequences(test_X, maxlen=max_len)

In [67]:
train_padded

array([[     0,      0,      0, ...,     87,  12477, 208250],
       [ 32113,    237,    891, ...,  24890,  30216,   1172],
       [     0,      0,      0, ...,    345,    976, 118902],
       ...,
       [     0,      0,      0, ...,  10370,   5724,  87112],
       [     0,      0,      0, ...,   1158,   2981,   2164],
       [     0,      0,      0, ...,    751,     39,    135]], dtype=int32)

In [61]:
print(train.FEATURE[0])
print(train_X[0])

pete cat bedtime blues doll 145 inchpete cat coolest popular cat town new pete cat bedtime blues doll merrymakers rocks striped pjs red slippers one sleepy cat ready cuddle measures 145 inches tall safe ages removable clothing surface wash new 2015pete cat bedtime blues plush dollbased popular pete cat books james deansuper cuddly ready naptime bedtimesafe ages perfect ages 2 upmeasures 14 5 inchesmerrymakers
[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0     

In [62]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 698155


In [68]:
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.fit_transform(test_labels)

In [ ]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove.twitter.27B.100d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [ ]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 100))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [49]:
model = Sequential()

model.add(
    Embedding(
        num_words,
        100,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=max_len,
        trainable=False,
    )
)
model.add(LSTM(150, dropout=0.1))
model.add(Dense(93, activation="Softmax"))


optimizer = Adam(learning_rate=3e-4)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [74]:
history = model.fit(
    train_padded,
    train_labels,
    epochs=5,
    validation_data=(test_padded, test_labels),
    verbose=1,
)